<a href="https://colab.research.google.com/github/ArnavGhosh999/INTERACTIVE-ARTWORK-GENERATOR/blob/main/Variational%20Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Flatten, Dense, Reshape, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import backend as K


In [ ]:
def load_images_from_folder(folder, img_size=(128, 128)):
    images = []
    for filename in os.listdir(folder):
        if filename.startswith('.'):  # Ignore hidden files and directories
            continue
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):  # Check if the path points to a file
            img = load_img(img_path, target_size=img_size)
            if img is not None:
                img_array = img_to_array(img)
                images.append(img_array)
    return np.array(images)

dataset_path = '/content/drive/MyDrive/artworks'
images = load_images_from_folder(dataset_path)
images = images.astype('float32') / 255.0  # Normalize the images

print(f"Number of images loaded: {len(images)}")


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Number of images loaded: 1134


In [ ]:
train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

In [ ]:
latent_dim = 100

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Encoder
inputs = Input(shape=(128, 128, 3))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
decoder_input = Input(shape=(latent_dim,))
x = Dense(32 * 32 * 128, activation='relu')(decoder_input)
x = Reshape((32, 32, 128))(x)
x = Conv2DTranspose(128, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
outputs = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
decoder = Model(decoder_input, outputs, name='decoder')

# VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae')

# VAE loss
reconstruction_loss = BinaryCrossentropy()(K.flatten(inputs), K.flatten(outputs))
reconstruction_loss *= 128 * 128 * 3
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')


In [ ]:
vae.fit(train_images, epochs=50, batch_size=32, validation_data=(val_images, None))


Epoch 1/50
29/29 [==============================] - 206s 7s/step - loss: 33279.0391 - val_loss: 32852.4141
Epoch 2/50
29/29 [==============================] - 205s 7s/step - loss: 32027.0430 - val_loss: 31790.7148
Epoch 3/50
29/29 [==============================] - 204s 7s/step - loss: 31542.9922 - val_loss: 31184.5215
Epoch 4/50
29/29 [==============================] - 199s 7s/step - loss: 31119.7793 - val_loss: 30759.3809
Epoch 5/50
29/29 [==============================] - 198s 7s/step - loss: 30425.5352 - val_loss: 30065.1250
Epoch 6/50
29/29 [==============================] - 211s 7s/step - loss: 29946.7090 - val_loss: 29650.5352
Epoch 7/50
29/29 [==============================] - 200s 7s/step - loss: 29567.9395 - val_loss: 29445.8398
Epoch 8/50
29/29 [==============================] - 201s 7s/step - loss: 29344.1855 - val_loss: 29367.8945
Epoch 9/50
29/29 [==============================] - 216s 7s/step - loss: 29222.2969 - val_loss: 29261.3516
Epoch 10/50
29/29 [==================